In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

import pymysql
import requests
import json
from datetime import datetime
import time
import logging

In [2]:
# 데이터 불러오기 및 필요한 함수 정의
base_src = 'Data'

# user 데이터
research_src = os.path.join(base_src, 'research_data_02.csv')
research = pd.read_csv(research_src,
                    sep=',',
                    encoding = 'cp949')
research = research.set_index('identifier')
research

,TIMESTAMP,1. 직업이 어떻게 되시나요?,2. 출생년도를 알려주세요!,3. 성별을 알려주세요!,4. 지난 한 달 동안 책을 몇 권 읽었나요?,5. 책을 읽는 시간은 하루에 얼마나 되나요?,6. 책을 읽고 내용을 기록하고 싶은 적이 있으셨나요?,7. 책을 읽고 내용에 대해 다른 사람과 이야기하고 싶은 적이 있으셨나요?,8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [책 01],8-1. 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요!( 평점 범위 : 0.0 ~ 5.0 ) [평점 01],...,8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 16],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 17],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 18],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 19],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [책 20],8-2 읽은 책들에 대한 평점을 남겨주세요! 많을수록 좋아요! ( 평점 범위 : 0.0 ~ 5.0 ) [평점 20],9. 기프티콘 추첨을 위해 번호를 입력해주세요!
identifier,,,,,,,,,,,,,,,,,,,,,
research044,2022-09-23 18:23,학생,2001,MALE,한 권도 읽지 않았다.,0시간,1,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1074639088
research045,2022-09-23 18:28,취업준비생,1999,FEMALE,1권 ~ 2권,0시간,4,4,로지컬씽킹의 기술,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
research046,2022-09-23 18:32,대학생,2001,FEMALE,3권 ~ 4권,2시간,4,4,포노사피엔스,4.5,...,4.5,왜 세계의 절반은 굶주리는가,4.0,돈으로 살 수 없는 것들,4.7,사랑 수업,5.0,나미야 잡화점의 기적,3.7,1053270166
research047,2022-09-23 18:37,학생,1999,MALE,1권 ~ 2권,1시간,2,3,숨결이 바람될때,5.0,...,5.0,열역학,1.0,기계요소설계,2.0,전기전자공학,3.0,교양글쓰기,2.0,1059412422
research048,2022-09-23 19:04,학생,1998,MALE,1권 ~ 2권,1시간,4,4,작별하지 않는다,3.5,...,4.0,그리고 아무도 없었다,3.5,죽음,2.0,데미안,4.5,코스모스,3.5,1056704424
research049,2022-09-23 19:05,학생,1998,MALE,1권 ~ 2권,1시간,4,4,마지막 몰입,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1021033096
research050,2022-09-23 20:01,대학생,2001,FEMALE,1권 ~ 2권,1시간,4,4,연어,4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1049973417
research051,2022-09-23 21:18,학생,1999,MALE,한 권도 읽지 않았다.,0시간,2,3,난쟁이기 쏘아올린 작은 공,5.0,...,4.0,만화로 쉽게 배우는 양자역학,4.0,만화로 쉽게 배우는 머신러닝,4.0,Brother One cell,3.5,Jade lady burning,4.0,010-5623-7535
research052,2022-09-23 22:28,학생,2002,FEMALE,1권 ~ 2권,1시간,1,2,"나는 풍요로웠고, 지구는 달라졌다",3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1046266990


In [3]:
#   db 연결
def dbconnect():
    conn = pymysql.connect (host='chaekin.cvg3ycl5pbuo.ap-northeast-2.rds.amazonaws.com', user='admin', password='chrlghk1!', db='testdb', charset='utf8')
    return conn

conn = dbconnect()  #   DB 연결

In [4]:
'''
member TB에서 identifier에 해당하는 row의 id를 가져옴
'''
def find_member_id(conn, identifier):
    cur = conn.cursor()
    sql = "SELECT id FROM member WHERE identifier = %s"
    cur.execute(sql, identifier)
    result = cur.fetchall()
    conn.commit()
    return result

In [5]:
'''
member TB에 member insert
'''
def insert_member(conn, member):
    cursor = conn.cursor()

    sql = "INSERT INTO member (age, gender, identifier, job, nickname) VALUES (%s, %s, %s, %s, %s)"
    
    cursor.execute(sql, member)
    conn.commit()

In [6]:
'''
research TB에 member_id, title, score insert
'''
def insert_research(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO research (member_id, title, score) VALUES (%s, %s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [7]:
'''
research TB에서 모든 record 가져오기
'''
def get_research_records(conn):
    cur = conn.cursor()
    sql = "SELECT * FROM research"
    cur.execute(sql)
    result = cur.fetchall()
    conn.commit()
    return result

In [8]:
'''
research TB에 isbn update
'''
def update_isbn(conn, item):
    cur = conn.cursor()
    sql = "update research set isbn = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [9]:
'''
research TB에 book_isbn_id update
'''
def update_book_isbn_id(conn, item):
    cur = conn.cursor()
    sql = "update research set book_isbn_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [10]:
'''
book_isbn TB에서 isbn에 해당하는 book_isbn_id 가져오기
'''
def get_book_isbn_id(conn, isbn):
    cur = conn.cursor()
    sql = "SELECT * FROM book_isbn where isbn = %s"
    cur.execute(sql,isbn)
    result = cur.fetchall()
    conn.commit()
    return result

In [11]:
'''
book_isbn TB에 isbn insert
'''
def insert_book_isbn(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO book_isbn (id, isbn) VALUES (%s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()

In [13]:
'''
research TB에서 모든 record 가져오기
'''
def search_title(title):
     #   요청 api 주소
    API_HEAD = 'http://www.aladin.co.kr/ttb/api/ItemSearch.aspx?ttbkey='
    #   api key
    API_KEYS = 'ttbbeckhem961036001'
    API_BODY = '&Query='
    API_TAIL = '&QueryType=Title&MaxResults=1&start=1&SearchTarget=Book&output=js&Version=20131101'

    url = API_HEAD + API_KEYS + API_BODY + title + API_TAIL
    res = requests.get(url)
    item = res.json()['item']
    if item:
        isbn = item[0]['isbn13']
    else:
        isbn = None
    
    return isbn

In [14]:
'''
booklog TB에 book_id, member_id, read_status insert
'''
def insert_book_isbn(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO booklog (book_id, member_id, read_status) VALUES (%s, %s, 'COMPLETE')"
    
    cursor.execute(sql, item)
    conn.commit()
    
    cursor = conn.cursor()

    sql = "select last_insert_id() from booklog"
    
    booklog_id = cursor.execute(sql)
    conn.commit()
    
    return booklog_id

In [18]:
'''
research TB에 booklog_id update
'''
def update_booklog_id(conn, item):
    cur = conn.cursor()
    sql = "update research set booklog_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [37]:
'''
review TB에 booklog_id, score insert 후 book score, count update
'''
def insert_review(conn, item):
    cursor = conn.cursor()

    sql = "INSERT INTO review (booklog_id, score) VALUES (%s, %s)"
    
    cursor.execute(sql, item)
    conn.commit()
    
    cursor = conn.cursor()

    sql = "select last_insert_id() from review"
    
    review_id = cursor.execute(sql)
    conn.commit()

    cur = conn.cursor()
    sql = "SELECT * FROM book WHERE id = (SELECT book_id FROM booklog WHERE id = %s);"
    cur.execute(sql, item[0])
    book = cur.fetchall()[0]
    conn.commit()
    
    score = book[10]
    count = book[11]
    
    update_count = count + 1
    update_score = (score * count + item[1])/update_count
    
    item2 = (update_score, update_count, book[0])
    
    cur = conn.cursor()
    sql = "update book set rating_score = %s, rating_count = %s where id = %s"
    cur.execute(sql, item2)
    conn.commit()
    
    return review_id

In [35]:
'''
research TB에 review_id update
'''
def update_review_id(conn, item):
    cur = conn.cursor()
    sql = "update research set review_id = %s where id = %s"
    cur.execute(sql, item)
    conn.commit()

In [ ]:
# 회원 정보 insert
for row in research.itertuples():
    data = (row[3], row[4], row[0], row[2], row[0])
    if len(find_member_id(conn, row[0])) == 0:
        insert_member(conn, data)
        print(data)

In [ ]:
# research TB에 member_id, title, score insert
for row in research.itertuples():
    member_id = find_member_id(conn, row[0])[0][0]
    print(member_id)
    for i in (range(9, 48, 2)):
        if pd.isna(row[i]):
            break
        print(row[i]," ",row[i+1])
        item = (member_id, row[i], row[i+1])
        insert_research(conn, item)

In [ ]:
# 책 이름으로 검색해서 isbn 받아오기
research_records = get_research_records(conn);
for row in research_records:
    print(row[4] == None)
    if row[4] == None:
        isbn = search_title(row[2])
        item = (isbn, row[0])
        if item:
            update_isbn(conn, item)
        print(item, row[2])

In [ ]:
# book_isbn에 추가
research_records = get_research_records(conn)
index = 74
for row in research_records:
    if row[5] != None:
        continue
    isbn = row[4]
    book_isbn_id = get_book_isbn_id(conn, isbn)
    if len(book_isbn_id) == 0:
        item = (index, isbn)
        insert_book_isbn(conn, item)
        book_isbn_id = get_book_isbn_id(conn, isbn)
        index = index + 1
    
    item2 = (book_isbn_id[0][0], row[0])
    print(item2)
    update_book_isbn_id(conn, item2)

In [21]:
# booklog에 추가
research_records = get_research_records(conn)
for row in research_records:
    if row[6] == None:
        item = (row[5], row[1])
        print(item)
        try:
            booklog_id = insert_book_isbn(conn, item)
            item2 = (booklog_id, row[0])
            update_booklog_id(conn, item2)
        except:
            print("error " , item2)

(5411220, 4)
error  (7, 7)
(5795911, 4)
(1323591, 4)
(1006570, 4)
(1072294, 4)
(1139434, 4)
(6310172, 5)
error  (12, 13)
(6310125, 5)
error  (12, 13)
(2, 5)
(3646824, 5)
error  (13, 16)
(5676032, 5)
(3278940, 5)
error  (14, 18)
(3, 5)
(5735384, 5)
(154527, 5)
(4, 5)
(5, 5)
(6, 5)
(3405992, 5)
error  (20, 25)
(5484919, 5)
(6326227, 5)
error  (21, 27)
(5400448, 5)
(5391271, 5)
(5747523, 5)
(6060889, 5)
error  (24, 31)
(6042837, 5)
error  (24, 31)
(7, 6)
(5408151, 6)
error  (25, 34)
(3754574, 6)
error  (25, 34)
(3441055, 6)
error  (25, 34)
(2376480, 6)
(2371850, 6)
(2373325, 6)
error  (27, 39)
(8, 6)
(2373084, 6)
(2370295, 6)
(3467245, 6)
error  (30, 43)
(2271872, 6)
(9, 6)
(6065686, 9)
error  (32, 46)
(3356410, 9)
error  (32, 46)
(6000477, 9)
error  (32, 46)
(10, 9)
(3748133, 9)
error  (33, 50)
(3687594, 9)
error  (33, 50)
(2619670, 9)
error  (33, 50)
(5385303, 9)
error  (33, 50)
(5236389, 9)
error  (33, 50)
(5236389, 9)
error  (33, 50)
(2645419, 9)
error  (33, 50)
(3877600, 9)
error  (3

(5375979, 54)
error  (320, 467)
(118, 54)
(3151708, 54)
error  (321, 470)
(3921678, 54)
error  (321, 470)
(119, 54)
(120, 54)
(5243813, 54)
error  (323, 474)
(5791228, 54)
(5967649, 54)
error  (324, 476)
(5849800, 54)
(5484946, 54)
(121, 54)
(6326227, 54)
error  (327, 480)
(2126974, 54)
(122, 54)
(5682111, 54)
(123, 54)
(124, 54)
(125, 54)
(5600419, 54)
(5795186, 54)
(126, 54)
(3494296, 55)
error  (336, 490)
(5854684, 55)
error  (336, 490)
(6066621, 55)
error  (336, 490)
(127, 55)
(128, 55)
(5438838, 56)
(129, 56)
(5735384, 56)
(130, 56)
(131, 56)
(1132126, 56)
(132, 56)
(133, 56)
(134, 56)
(6325528, 56)
error  (347, 504)
(135, 56)
(136, 56)
(137, 56)
(138, 56)
(139, 56)
(140, 56)
(141, 56)
(142, 56)
(143, 56)
(144, 56)
(6036502, 57)
error  (357, 515)
(145, 57)
(5447803, 57)
error  (358, 517)
(146, 57)
(3107992, 57)
error  (359, 519)
(147, 57)
(5354680, 57)
error  (360, 521)
(5242622, 57)
error  (360, 521)
(148, 57)
(5392430, 57)
error  (361, 524)
(149, 57)
(150, 57)
(3408932, 57)
erro

In [40]:
# review에 추가 & book score, count update
research_records = get_research_records(conn)
for row in research_records:
    print(row[6])
    if row[6] != None and row[7] == None:
        item = (row[6], row[3])
        review_id = insert_review(conn, item)
        item2 = (row[0], review_id)
        update_review_id(conn, item2)
        print(item, item2)

1
2
3
4
5
6
7
None
10
(10, 3.5) (9, 8)
11
(11, 3.0) (10, 9)
12
(12, 4.0) (11, 10)
13
(13, 4.0) (12, 11)
14
(14, 3.5) (13, 12)
None
None
17
(17, 3.8) (16, 13)
None
19
(19, 3.0) (18, 14)
None
21
(21, 4.2) (20, 15)
22
(22, 4.6) (21, 16)
23
(23, 3.9) (22, 17)
24
(24, 4.1) (23, 18)
25
(25, 4.6) (24, 19)
26
(26, 4.7) (25, 20)
None
28
(28, 4.2) (27, 21)
None
30
(30, 3.9) (29, 22)
31
(31, 4.4) (30, 23)
32
(32, 4.5) (31, 24)
None
None
35
(35, 4.0) (34, 25)
None
None
None
39
(39, 4.5) (38, 26)
40
(40, 4.0) (39, 27)
None
42
(42, 5.0) (41, 28)
43
(43, 4.8) (42, 29)
44
(44, 4.5) (43, 30)
None
46
(46, 4.4) (45, 31)
47
(47, 4.0) (46, 32)
None
None
None
51
(51, 4.5) (50, 33)
None
None
None
None
None
None
None
None
None
None
62
(62, 4.5) (61, 34)
63
(63, 3.5) (62, 35)
None
None
66
(66, 3.5) (65, 36)
None
68
(68, 5.0) (67, 37)
69
(69, 5.0) (68, 38)
70
(70, 4.0) (69, 39)
71
(71, 4.0) (70, 40)
72
(72, 5.0) (71, 41)
73
(73, 5.0) (72, 42)
74
(74, 3.5) (73, 43)
None
None
None
None
None
80
(80, 3.5) (79, 44)


(450, 3.0) (450, 306)
451
(451, 2.0) (451, 307)
452
(452, 5.0) (452, 308)
None
454
(454, 3.0) (454, 309)
455
(455, 4.0) (455, 310)
456
(456, 3.0) (456, 311)
457
(457, 3.0) (457, 312)
458
(458, 4.0) (458, 313)
459
(459, 3.0) (459, 314)
460
(460, 4.0) (460, 315)
461
(461, 4.0) (461, 316)
None
463
(463, 4.0) (463, 317)
464
(464, 5.0) (464, 318)
None
466
(466, 4.0) (466, 319)
467
(467, 5.0) (467, 320)
None
469
(469, 4.0) (470, 321)
None
None
472
(472, 4.0) (473, 322)
473
(473, 5.0) (474, 323)
None
475
(475, 5.0) (476, 324)
None
477
(477, 4.0) (478, 325)
478
(478, 5.0) (479, 326)
479
(479, 4.0) (480, 327)
None
481
(481, 3.0) (482, 328)
482
(482, 4.0) (483, 329)
483
(483, 5.0) (484, 330)
484
(484, 4.0) (485, 331)
485
(485, 5.0) (486, 332)
486
(486, 3.0) (487, 333)
487
(487, 2.0) (488, 334)
488
(488, 4.0) (489, 335)
489
(489, 4.0) (490, 336)
None
None
None
493
(493, 4.8) (494, 337)
494
(494, 4.5) (495, 338)
495
(495, 5.0) (496, 339)
496
(496, 5.0) (497, 340)
497
(497, 5.0) (498, 341)
498
(498